In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision.models as models
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import cv2
import os
import numpy as np
import re
from tqdm import tqdm

/Users/ianzou/opt/anaconda3/envs/torch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def hist(image):
    b, g, r = cv2.split(image)
    r1 = cv2.equalizeHist(r)
    g1 = cv2.equalizeHist(g)
    b1 = cv2.equalizeHist(b)
    image_equal_clo = cv2.merge([r1, g1, b1])
    return image_equal_clo


pipline = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize([224, 224]),
        transforms.Grayscale(3),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[
                             0.229, 0.224, 0.225])
        #transforms.Normalize((0.1307, 0.1307, 0.1307),(0.3081, 0.3081, 0.3081))

    ]
)

In [9]:
# 迁移学习ResNet结构定义

num_classes = 500
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, num_classes)
)

for p in model.parameters():
    p.requires_grad = False
for layer in [model.layer4.parameters(), model.fc.parameters()]:
    for p in layer:
        p.requires_grad = True

In [10]:
model.load_state_dict(torch.load('/Users/ianzou/Desktop/Cornorstone/机器学习/大作业_掌静脉识别/resnet_final.pth', map_location='cpu'))
# model.to(DEVICE)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [14]:
path1 = '/Users/ianzou/Desktop/Cornorstone/机器学习/大作业_掌静脉识别/all/'
dbtype_list = os.listdir(path1)
dbtype_list.pop(dbtype_list.index('.DS_Store'))
pred_label = []
pred_set1 = []
pred_set2 = []
for i in range(len(dbtype_list)):
    # if dbtype_list[i][:5] == 'false':
    #     pred_label.append(0)
    # else:
    #     pred_label.append(1)
    pic_name = os.listdir(path1+dbtype_list[i])
    for j in range(len(pic_name)):
        if pic_name[j][:5] == 'first':
            img = cv2.imread(path1 + dbtype_list[i] + '/' + pic_name[j])
            img = hist(img)
            pred_set1.append(pipline(img).cpu().numpy().tolist())
            pred_label.append(int(dbtype_list[i])-1)
        else:
            img = cv2.imread(path1 + dbtype_list[i] + '/' + pic_name[j])
            img = hist(img)
            pred_set2.append(pipline(img).cpu().numpy().tolist())

test_set_m1 = TensorDataset(torch.tensor(pred_set1))
test_set_m2 = TensorDataset(torch.tensor(pred_set2))

In [20]:
fpr_list = []
tpr_list = []
auc_score = []
suss_list = []


# theta = 0.7
for theta in [0.9921]:
    cor = 0
    wro_acc = 0
    wro_ref = 0
    ref = 0
    real = []
    predict = []
    suss = []
    for i in tqdm(range(len(test_set_m1))):
        t1 = test_set_m1[i][0].to(DEVICE).unsqueeze(0).float()
        t2 = test_set_m2[i][0].to(DEVICE).unsqueeze(0).float()
        t1_output = model(t1)
        t2_output = model(t2)
        t1_compare = t1_output.reshape(t1_output.shape[0]*t1_output.shape[1])
        t1_compare = t1_compare.unsqueeze(0).float()
        t2_compare = t2_output.reshape(t2_output.shape[0]*t2_output.shape[1])
        t2_compare = t2_compare.unsqueeze(0).float()
        simi = torch.cosine_similarity(t1_compare, t2_compare)
        # simi = F.pairwise_distance(t1_compare, t2_compare)
        # simi = np.corrcoef(t1_compare.cpu().detach().numpy(), t2_compare.cpu().detach().numpy())[0][-1]
        # print(simi)
        if simi >= theta:
            predict.append(1)
        else:
            predict.append(0)

    # fpr, tpr, thersholds = roc_curve(real, predict)
    # fpr_list.append(fpr)
    # tpr_list.append(tpr)
    # auc_score.append(auc(fpr, tpr))
    # suss_list.append(np.sum(suss))

100%|██████████| 1000/1000 [03:31<00:00,  4.73it/s]


In [21]:
predict

[1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,


In [19]:
for i in range(len(pred_label)):
    pred_label[i] -= 1

1000

In [1]:
import pandas as pd

In [24]:
list2 = []
for i in range(len(pred_label)):
    list2.append([pred_label[i], predict[i]])

list2

[[797, 1],
 [909, 1],
 [135, 1],
 [307, 0],
 [763, 0],
 [551, 1],
 [61, 0],
 [95, 0],
 [338, 0],
 [936, 1],
 [556, 1],
 [764, 0],
 [300, 1],
 [132, 0],
 [59, 0],
 [790, 1],
 [931, 0],
 [569, 1],
 [92, 0],
 [66, 1],
 [336, 1],
 [938, 0],
 [104, 0],
 [560, 0],
 [752, 0],
 [594, 0],
 [907, 0],
 [309, 1],
 [799, 0],
 [50, 0],
 [68, 1],
 [593, 0],
 [755, 0],
 [567, 0],
 [103, 0],
 [331, 1],
 [57, 0],
 [900, 1],
 [558, 1],
 [168, 0],
 [954, 1],
 [391, 0],
 [533, 1],
 [701, 0],
 [365, 1],
 [157, 1],
 [739, 1],
 [953, 1],
 [150, 0],
 [362, 1],
 [706, 1],
 [534, 1],
 [998, 0],
 [396, 1],
 [159, 1],
 [965, 0],
 [32, 0],
 [991, 0],
 [730, 0],
 [502, 0],
 [166, 0],
 [354, 1],
 [192, 0],
 [996, 1],
 [398, 0],
 [35, 1],
 [708, 0],
 [962, 0],
 [195, 1],
 [353, 0],
 [161, 1],
 [505, 1],
 [737, 1],
 [566, 0],
 [754, 1],
 [330, 0],
 [102, 0],
 [592, 0],
 [69, 0],
 [901, 1],
 [559, 1],
 [56, 0],
 [595, 1],
 [105, 1],
 [939, 1],
 [337, 1],
 [753, 0],
 [561, 1],
 [51, 0],
 [798, 0],
 [308, 1],
 [906, 1],
 

In [26]:
list = [pred_label, predict]
test = pd.DataFrame(data=list2, index=None)
test

,0,1
0,797,1
1,909,1
2,135,1
3,307,0
4,763,0
...,...,...
995,972,0
996,718,1
997,25,0
998,388,1


In [28]:
test.sort_values(by=0, inplace=True, ascending=True)

In [29]:
test

,0,1
440,1,1
966,2,0
915,3,0
908,4,1
972,5,1
...,...,...
63,996,1
115,997,1
52,998,0
134,999,1


In [30]:
test.to_csv('/Users/ianzou/Desktop/2019193009.txt', sep='\t', index=False)